In [ ]:
# Python ≥3.9 is required
import sys
assert sys.version_info >= (3, 9)


# Common imports
import numpy as np
import scipy as sc

# Import helper functions
from helper_functions import *

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('figure', figsize=(16,6))

### Exploratory Analysis

In [ ]:
# Load PPG and ECG signals
signal_ppg_1, signal_ppg_2, signal_ppg_3 = load_signals_ppg()
signal_ecg_1, signal_ecg_2, signal_ecg_3 = load_signals_ecg()

In [ ]:
# Plot PPG signals
plot_all_ppg_signals(signal_ppg_1, signal_ppg_2, signal_ppg_3)

In [ ]:
# Plot PPG signals between sample 540 and 600
plot_all_ppg_signals(signal_ppg_1, signal_ppg_2, signal_ppg_3, 540, 600)

In [ ]:
# Plot ECG signals
plot_all_ecg_signals(signal_ecg_1, signal_ecg_2, signal_ecg_3)

In [ ]:
# Plot ECG signals between sample 20k and 22k
plot_all_ecg_signals(signal_ecg_1, signal_ecg_2, signal_ecg_3, 20000, 22000)

### Sampling Rate

A person at rest has a heart rate of 60 to 100 bpm. For healthy people this value is around 60 bpm.
For simplicity we will consider 60 bpm for a healthy person at rest, i.e. we have 60 pulses or peaks in a 1-minute interval or 1 pulse/peak per second.

We proceed as follows. For PPG signals we locate a pulse by inspection, compute the local minima and assure that the distance between two minima is greater 20 samples (value obtained by inspection). The distance between these two minima then gives us the pulse width in samples and inferes the sampling frequency. It turns out that the sample between 540 and 600 is sufficient to locate two pulses (interval hard coded in code) for all three PPG signals.

For the case of ECG signals, we locate two pulses (or rather two peaks) and compute the distance between these two maxima which gives us the duration of one period (due to periodicity of the pulses) in samples. In this scenario, the the interval between 20k and 22k is sufficient.

In [ ]:
get_sampling_rate(signal_ppg_1, 'ppg', 1)
get_sampling_rate(signal_ppg_2, 'ppg', 2)
get_sampling_rate(signal_ppg_3, 'ppg', 3)
get_sampling_rate(signal_ecg_1, 'ecg', 1)
get_sampling_rate(signal_ecg_2, 'ecg', 2)
get_sampling_rate(signal_ecg_3, 'ecg', 3)

We observe that the PPG and ECG signals were roughly sampled with the same frequency. In the remainder of the tasks we consider a sampling frequency of fs = 25 Hz (ts = 40ms) and fs = 1000 Hz (ts = 1ms) for PPG and ECG signals, repsectively.

In [ ]:
fs_ppg = 25
fs_ecg = 1000
ts_ppg = 0.04
ts_ecg = 0.001

### Signal Processing

In each signal we observe high frequency noise, trend and offset. High frequency noise can be mitigated by using a lowpass filter. Observing the trend, we see that it is not a linear trend. One way to acount for the trend would be using a polynomial to fit the data points and subtract the fitted data points from the originals. This is not ideal, especially if we use higher degree polynomials. Higher degree polynomials might fit the data, but will also follow the noise pattern. Instead, we will filter out lower frequencies (e.g. with a highpass filter) that have an impact on the trend.

Since we want to both filter out lower and higher frequencies, we will consider a bandpass filter. A decent choice here would be a Butterworth filter of order 5. To get sensible values for the cutoff frequencies, one has to first inspect the signal spectrum.

In [ ]:
# Initialize all signals
sig_ppg_1 = Signal(signal_ppg_1, 'ppg', 1)
sig_ppg_1.initialize()

sig_ecg_1 = Signal(signal_ecg_1, 'ecg', 1)
sig_ecg_1.initialize()

sig_ppg_2 = Signal(signal_ppg_2, 'ppg', 2)
sig_ppg_2.initialize()

sig_ecg_2 = Signal(signal_ecg_2, 'ecg', 2)
sig_ecg_2.initialize()

sig_ppg_3 = Signal(signal_ppg_3, 'ppg', 3)
sig_ppg_3.initialize()

sig_ecg_3 = Signal(signal_ecg_3, 'ecg', 3)
sig_ecg_3.initialize()

Plot spectrum of all ECG and PPG signals

In [ ]:
sig_ppg_1.plot_signal_spectrum()

In [ ]:
sig_ecg_1.plot_signal_spectrum()

In [ ]:
sig_ppg_2.plot_signal_spectrum()

In [ ]:
sig_ecg_2.plot_signal_spectrum()

In [ ]:
sig_ppg_3.plot_signal_spectrum()

In [ ]:
sig_ecg_3.plot_signal_spectrum()

By looking at the signal spectra, we define the following cutoff frequencies. For PPG signals we set
0.5 Hz for the lower and 1.5 Hz for the higher cutoff frequency whereas for ECG signals these values are set to 0.5 Hz and 50 Hz, respectively.

In [ ]:
# Preprocess filter by using a Butterworth filter of order 5.
sig_ppg_1.preprocess_signal()
sig_ecg_1.preprocess_signal()

sig_ppg_2.preprocess_signal()
sig_ecg_2.preprocess_signal()

sig_ppg_3.preprocess_signal()
sig_ecg_3.preprocess_signal()

In [ ]:
# Plot preprocessed signals
sig_ppg_1.plot_signal()
sig_ecg_1.plot_signal()

In [ ]:
sig_ppg_2.plot_signal()
sig_ecg_2.plot_signal()

In [ ]:
sig_ppg_3.plot_signal()
sig_ecg_3.plot_signal()

We managed to remove trend, high frequency noise and offset, although it is not perfect.

### Peak Detection

For peak detection, we use the find_peaks function from the scipy library.

In [ ]:
# Get peak indices
sig_ppg_1.get_peak_indices()
sig_ecg_1.get_peak_indices()

sig_ppg_2.get_peak_indices()
sig_ecg_2.get_peak_indices()

sig_ppg_3.get_peak_indices()
sig_ecg_3.get_peak_indices()

In [ ]:
# Plot peaks

In [ ]:
sig_ppg_1.plot_peaks()
sig_ecg_1.plot_peaks()

In [ ]:
sig_ppg_2.plot_peaks()
sig_ecg_2.plot_peaks()

In [ ]:
sig_ppg_3.plot_peaks()

### Anomaly Detection in PPG signals

For anomaly or outlier detection we could use a moving average filter our go even further and use an autoregressive integrated moving average (ARIMA) filter. We here, however, take a different approach.

We will simply compute the prominences of the peaks which then gives us the pulse height. If a peak is above or below a certain median threshold, we  consider it an anomaly. If two or more succeeding prominences lay outside the threshold regime, we define it as an anomaly segment.


In [ ]:
# Get peak prominences for all PPG signals
sig_ppg_1.get_peak_prominences()
sig_ppg_2.get_peak_prominences()
sig_ppg_3.get_peak_prominences()

In [ ]:
# Plot PPG signal 1 with prominences
sig_ppg_1.plot_peak_prominences()


In [ ]:
# Detect anomalies
sig_ppg_1.detect_anomalies()
sig_ppg_2.detect_anomalies()
sig_ppg_3.detect_anomalies()

In [ ]:
# Plot anomalies
sig_ppg_1.plot_anomalies()
sig_ppg_2.plot_anomalies()
sig_ppg_3.plot_anomalies()

In [ ]:
# Plot outliers and anomaly segments
sig_ppg_1.plot_anomaly_segments()
sig_ppg_2.plot_anomaly_segments()
sig_ppg_3.plot_anomaly_segments()

The red-shaded segments describe the anomaly segments whereas the black 'X' marks define the outliers.

This approach is not ideal, but at least gives some sensitive results. By inspection, PPG signal 1 actually has the best quality among all three PPG signals. In fact, we should not expect any outliers or anomalies. Yet, our approach introduces some detection errors which fortunately are not severe.

### Synchronization

To synchronize PPG and ECG signals, both signals need to have the same sampling rate. We have seen earlier that this is not the case. In fact, ECG signals were sampled with a rate that is 40 (1000 Hz / 25 Hz) times higher than the sampling frequency of the PPG signals. We thus downsample the ECG signals first and then compute correlation between two signals. The max correlation point then gives us the "lag", i.e. the point or number of samples that one signal has to be shifted in order to synchronize both signals.

In [ ]:
# Compute lag
sig_ppg_1.compute_lag(sig_ecg_1)
sig_ppg_2.compute_lag(sig_ecg_2)
sig_ppg_3.compute_lag(sig_ecg_3)

We observe that the for PPG and ECG signal 1, the lag seems to be fine. For the other two signals, the value is a bit too high. One of the reasons might be that were are not catching the right points by downsampling the ECG signals and are basically discarding important values.

### Evaluation

One way to evaluate our signals would be by synchronizing our signals (PPG X and ECG X) and then compute the root mean square error (RMSE) where a lower RMSE accounts for higher quality. However, due to above issues, we will consider a simpler approach.

Each PPG signal will be given a score S between 1 and 0. The score is calculated as S = 1 - #anomlies/#peaks.
A higher score is proportional to a better signal quality.

In [ ]:
# Compute scores
sig_ppg_1.get_score()
sig_ppg_2.get_score()
sig_ppg_3.get_score()

In [ ]:
# Score of PPG signal 1
sig_ppg_1.score

In [ ]:
# Score of PPG signal 2
sig_ppg_2.score

In [ ]:
# Score of PPG signal 3
sig_ppg_3.score

Just by looking at raw data, one might think that signal 2 has a higher quality than signal 3. Yet, processing the signals allows us to get a better insight.

Our metric ranks the PPG signals as follows (from best to worse):

PPG Signal 1

PPG Signal 3

PPG Signal 2